<a href="https://colab.research.google.com/github/SridharSeshadri56/Decision_Models/blob/main/pyomoDemoAnswerExercise2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import * works when algebraic formulation using sets. Else use the import as pyo

In [1]:
pip install pyomo  #Installs the modeling language called pyomo

     |████████████████████████████████| 9.6 MB 6.5 MB/s 
     |████████████████████████████████| 49 kB 2.1 MB/s 


In [2]:
# The GLPK (GNU Linear Programming Kit) package is intended for solving large-scale linear programming (LP), 
# mixed integer programming (MIP), and other related problems. It is a set of routines written in ANSI C and 
# organized in the form of a callable library.
!apt-get install -y -qq glpk-utils  #Installs the optimization engine called glpk.


Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 155320 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Setting up libcolamd2:amd64 (1:5.1.2-2) ...
Setting up libamd2:amd64 

In [ ]:
from pyomo.environ import *              # Some what sloppy but makes writing faster
A = ['Astros', 'Cosmos']                 # A is the set of names of decision variables
profit = {'Astros':20, 'Cosmos':30}      # profit is the profit from each product
w1 = {'Astros':1, 'Cosmos':2}            # These are the hours needed in picture tube
w2 = {'Astros':1, 'Cosmos':1}            # These are the hours need in chassis assmebly
Picture_Tube_Max = 120                   # Hours available in picture tube
Chassis_Assembly_Max = 95                # Hours available in chassis assembly

model = ConcreteModel(name = "(Model1)")      # This is called a ConcreteModel becaue data is given along with model (else use abstract model)
                                              # You can call this instead of model, like Mod, Md, problem, etc
model.x = Var( A, within= NonNegativeReals )  # This defines the variables - how much to produce. You can calusel any name, here "x"
                                              # Declares the "x" to  be NonNegativeReals, that is, >= 0 and continuous (real) [like solver]

# This defines the objective. Has expression for the objective, "expr". And "sense" states whether maximize or minimize (default).
# Uses the keyword "Objective" (like the solver)
model.value = Objective(                      
expr = sum( profit[i]*model.x[i] for i in A), sense = maximize )  # Note indexes over names of decision variables.

# Defines the picture tube production hours constraint. Note uses the keyword "Constraint" (like solver)
model.picture_tube = Constraint(                
expr = sum( w1[i]*model.x[i] for i in A) <=  Picture_Tube_Max)    

 # Defines the chassis_assembly production hours constraint. Need to write each constraint.
model.chassis_assembly = Constraint(           
expr = sum( w2[i]*model.x[i] for i in A) <= Chassis_Assembly_Max )

opt = SolverFactory('glpk')                     # Choose the optimization engine (like in excel we choose Simplex, GRG etc)

model.dual = Suffix(direction=Suffix.IMPORT_EXPORT)  # This states report the shadow prices (called dual)

results = opt.solve(model, tee= True)           # Calls the optimizer. Tee = True says provide output. Else silent. We can print later

In [ ]:
model.x.pprint()   # Prints the decision variables

In [ ]:
model.pprint()   # prints the entire output (note gives shadow prices at very end). Does not give allowed ranges :)

Exercise 1: Add the two missing constraints, Astros <= 70; Cosmos <= 50.

# Answer: We need to add weights for two constraints:
w3 = {'Astros':1, 'Cosmos':0}  # This is for Astros circuit boards

w4 = {'Astros':0, 'Cosmos':1}  # This is for Cosmos circuit boards

# Two new variables: 
Astros_circuit_board_max = 70  # Available Astros circuit boards

Cosmos_circuit_board_max = 50  # Available Cosmos circuit boars

# Then two new constraints:
# Defines the circuit board constraint for Astros
model.Astros_circuit_boards = Constraint(                
expr = sum( w3[i]*model.x[i] for i in A) <=  Astros_circuit_board_max)    

 # Defines the Cosmos circuit board constraint.
model.Cosmos_circuit_boards = Constraint(           
expr = sum( w4[i]*model.x[i] for i in A) <= Cosmos_circuit_board_max )

# Thats it! Try it.

We will next make the declaration of constraints easier by defining "rule"

In [4]:

A = ['Astros', 'Cosmos']                   # A is the set of names of decision variables
profit = {'Astros':20, 'Cosmos':30}        # profit from each product

constraints = {'Picture_Tube', 'Chassis_Assembly'}  # The set of constraints (this is new to this method)

 # We are defining by constraint the coefficient of each variable
coefficients = { ('Picture_Tube','Astros'):1, \
             ('Picture_Tube', 'Cosmos'):2, \
             ('Chassis_Assembly','Astros'):1, \
             ('Chassis_Assembly','Cosmos'):1}

capacity = {'Picture_Tube': 120, 'Chassis_Assembly': 96}   # This again is new because we define the right hand side by constraint

model = ConcreteModel(name = "(Model2)")           # Same as previous
model.x = Var( A, within= NonNegativeReals )       # Decision variables as previous
model.value = Objective(                           # Objective, same as previous
expr = sum( profit[i]*model.x[i] for i in A), sense = maximize )

# This defines a rule called one per constraint. Basically how to compute the left hand side of each constraint.
# m stands for model (will get substituted when we call the rule. c stands for the name of the constraint, again substituted when called)
def one_per_constraint_rule(m,c):
    return sum(m.x[a]*coefficients[c,a] for a in A) <= capacity [c]    # sums hours required for constraint c and sets <= capacity for c.

# This defines in our model the constraints! Note that we simply pass the set of constraints and the rule. It does the rest.
# model is by default when we call (recall model can be renamed as you like )
model.one_per_constraint = Constraint(constraints, rule = one_per_constraint_rule)

opt = SolverFactory('glpk')           # same as before

model.dual = Suffix(direction=Suffix.IMPORT_EXPORT)   # same as before
results = opt.solve(model, tee= True)                 # same as before

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /tmp/tmpih1eoc2r.glpk.raw --wglp /tmp/tmpqr2w1aqf.glpk.glp --cpxlp
 /tmp/tmpebpbc24n.pyomo.lp
Reading problem data from '/tmp/tmpebpbc24n.pyomo.lp'...
3 rows, 3 columns, 5 non-zeros
26 lines were read
Writing problem data to '/tmp/tmpqr2w1aqf.glpk.glp'...
19 lines were written
GLPK Simplex Optimizer, v4.65
3 rows, 3 columns, 5 non-zeros
Preprocessing...
2 rows, 2 columns, 4 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  2.000e+00  ratio =  2.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 2
*     0: obj =  -0.000000000e+00 inf =   0.000e+00 (2)
*     2: obj =   2.160000000e+03 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.0 Mb (40412 bytes)
Writing basic solution to '/tmp/tmpih1eoc2r.glpk.raw'...
15 lines were written


In [5]:
model.pprint()

2 Set Declarations
    one_per_constraint_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'Chassis_Assembly', 'Picture_Tube'}
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'Astros', 'Cosmos'}

1 Var Declarations
    x : Size=2, Index=x_index
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        Astros :     0 :  72.0 :  None : False : False : NonNegativeReals
        Cosmos :     0 :  24.0 :  None : False : False : NonNegativeReals

1 Objective Declarations
    value : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 20*x[Astros] + 30*x[Cosmos]

1 Constraint Declarations
    one_per_constraint : Size=2, Index=one_per_constraint_index, Active=True
        Key              : Lower : Body                    : Upper : Active
        Chass

Exercise 2: Add the two constraints. Astros <= 70; Cosmos <= 50.

#Answer

Add the names of new constraints to the set of constraints:

 constraints = {'Picture_Tube', 'Chassis_Assembly', 'Astros_ciruit_boards', 'Cosmos_circuit_boards' }  

Add the coefficents of the decision variables in the set of coefficients:

coefficients = { ('Picture_Tube','Astros'):1, \
             ('Picture_Tube', 'Cosmos'):2, \
             ('Chassis_Assembly','Astros'):1, \
             ('Chassis_Assembly','Cosmos'):1, \
             ('Astros_ciruit_boards', 'Astros'): 1, \
             ('Astros_ciruit_boards', 'Cosmos'): 0, \
             ('Cosmos_circuit_boards', 'Astros'):0, \
             ('Cosmos_circuit_boards', 'Cosmos'):1 }

Add the capacity for the two new constraints in the set of capacity:

capacity = {'Picture_Tube': 120, 'Chassis_Assembly': 96, \
'Astros_ciruit_boards': 70, 'Cosmos_circuit_boards':50}

Solve and see!

We will discuss the Knapsack model later.

In [ ]:
# The Knapsack Model

In [ ]:
import pyomo.environ as pyo
model = pyo.ConcreteModel()
model.x = pyo.Var([1,2], domain=pyo.NonNegativeReals)
model.OBJ = pyo.Objective(expr = 2*model.x[1] + 3*model.x[2])
model.Constraint1 = pyo.Constraint(expr = 3*model.x[1] + 4*model.x[2] >= 2)
model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT_EXPORT)

In [ ]:
from pyomo.environ import *
A = ['hammer', 'wrench', 'screwdriver', 'towel']
b = {'hammer':8, 'wrench':3, 'screwdriver':6, 'towel':11}
w = {'hammer':5, 'wrench':7, 'screwdriver':4, 'towel':3}
W_max = 14
model = ConcreteModel()
model.x = Var( A, within=Binary )
model.value = Objective(
expr = sum( b[i]*model.x[i] for i in A),
sense = maximize )
model.weight = Constraint(
expr = sum( w[i]*model.x[i] for i in A) <= W_max )


In [ ]:
opt = SolverFactory('glpk')


In [ ]:
model.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
results = opt.solve(model, tee= True)

In [ ]:
model.x.pprint()

In [ ]:
model.pprint()

In [ ]:
model.x.pprint()

In [ ]:
print (model.dual[model.Constraint1])   # prints by defining and retrieving dual by constraint

In [ ]:
x1 = model.weight.uslack()
x2 = model.weight.lslack()

In [ ]:
print ("Upper Slack = ", x1, ", Lower slack =", x2)